In [3]:
import optuna
import pandas as pd
import numpy as np
import datetime
import os
from stable_baselines3 import A2C
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

# 🧠 Set up Optuna storage
os.makedirs("optuna_logs_", exist_ok=True)
db_path = "sqlite:///optuna_logs_/a2c_optuna_study.db"

# Load dataset
df = pd.read_csv("cleaned_file.csv")
df['date'] = pd.to_datetime(df['date'])

# Filter AAPL
df = df[df['tic'] == 'AAPL'].sort_values(by='date')
print(f"Overall AAPL data range: {df['date'].min()} to {df['date'].max()}")
print(f"Total AAPL rows: {len(df)}")

# Tech indicators
tech_indicator_list = [
    'macd', 'rsi_30', 'cci_30', 'dx_30',
    'close_30_sma', 'close_60_sma',
    'boll_ub', 'boll_lb','vix', 'turbulence'
]

# Create stock trading environment
def create_env(data):
    data = data.reset_index(drop=True)
    print(f"Inside create_env: data shape {data.shape}, index range {data.index.min()} to {data.index.max()}")

    stock_dim = len(data.tic.unique())
    state_space = 1 + 2 * stock_dim + len(tech_indicator_list) * stock_dim
    action_space = stock_dim
    num_stock_shares = [0] * stock_dim

    return StockTradingEnv(
        df=data,
        stock_dim=stock_dim,
        hmax=100,
        initial_amount=1e6,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=[0.001],
        sell_cost_pct=[0.001],
        reward_scaling=1e-4,
        state_space=state_space,
        action_space=action_space,
        tech_indicator_list=tech_indicator_list,
        user_defined_feature=[]
    )

# Helper to serialize params to a hashable tuple
def serialize_params(params):
    return tuple(sorted(params.items()))

# Set to track tried params during this run
tried_params = set()

# Optuna objective function
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True),
        'n_steps': trial.suggest_categorical('n_steps', [5, 10, 20, 50]),
        'gamma': trial.suggest_float('gamma', 0.9, 0.999),
        'ent_coef': trial.suggest_float('ent_coef', 1e-8, 1e-2, log=True)
    }

    param_key = serialize_params(params)
    if param_key in tried_params:
        # Duplicate parameters found, prune this trial
        raise optuna.exceptions.TrialPruned()
    tried_params.add(param_key)

    learning_rate = params['learning_rate']
    n_steps = params['n_steps']
    gamma = params['gamma']
    ent_coef = params['ent_coef']

    # Manual date split
    train_df = df[(df['date'] >= '2020-02-02') & (df['date'] <= '2022-12-31')].reset_index(drop=True)
    test_df  = df[(df['date'] >= '2023-01-01') & (df['date'] <= '2023-12-28')].reset_index(drop=True)

    print(f"Train df shape: {train_df.shape}, index range: {train_df.index.min()} to {train_df.index.max()}")
    print(f"Test df shape: {test_df.shape}, index range: {test_df.index.min()} to {test_df.index.max()}")

    if train_df.empty or test_df.empty:
        print("Warning: Train or test data is empty for these date ranges.")
        return -float('inf')

    env_train = create_env(train_df)
    env_test = create_env(test_df)

    model = A2C(
        "MlpPolicy",
        env_train,
        learning_rate=learning_rate,
        n_steps=n_steps,
        gamma=gamma,
        ent_coef=ent_coef,
        verbose=0,
        device="cpu"
    )

    model.learn(total_timesteps=10000)

    # Evaluation
    obs, _ = env_test.reset()
    total_reward = 0
    done = False

    while not done:
        action, _ = model.predict(obs)
        obs, reward, terminated, truncated, info = env_test.step(action)
        done = terminated or truncated

        total_reward += reward
        print(f"Step reward: {reward}")
        print(f"Current state shape: {np.array(obs).shape}")

    print(f"Total test reward: {total_reward}")
    print(f"[I {datetime.datetime.now()}] Trial {trial.number} finished with value: {total_reward} and parameters: {trial.params}")

    return total_reward

# Run tuning with storage
study = optuna.create_study(
    direction='maximize',
    study_name='a2c_hyperparam_tuning',
    storage=db_path,
    load_if_exists=True
)
study.optimize(objective, n_trials=20)

# Results
print("Best parameters:", study.best_params)
print("Best reward:", study.best_value)


Overall AAPL data range: 2020-01-02 00:00:00 to 2023-12-28 00:00:00
Total AAPL rows: 1013


[I 2025-07-03 14:38:11,043] A new study created in RDB with name: a2c_hyperparam_tuning


Train df shape: (743, 18), index range: 0 to 742
Test df shape: (249, 18), index range: 0 to 248
Inside create_env: data shape (743, 18), index range 0 to 742
Inside create_env: data shape (249, 18), index range 0 to 248
state: [1000000.0, np.float64(74.63008117675781), 0, np.float64(0.2504165059637273), np.float64(51.398809784158985), np.float64(-34.20001240594045), np.float64(11.97406829413876), np.float64(75.38160601529208), np.float64(75.38160601529208), np.float64(79.16018485227742), np.float64(72.22578392824015), np.float64(17.969999313354492), np.float64(0.0)]
np.array(state).shape: (13,)
state: [1000000.0, np.float64(74.63008117675781), 0, np.float64(0.2504165059637273), np.float64(51.398809784158985), np.float64(-34.20001240594045), np.float64(11.97406829413876), np.float64(75.38160601529208), np.float64(75.38160601529208), np.float64(79.16018485227742), np.float64(72.22578392824015), np.float64(17.969999313354492), np.float64(0.0)]
np.array(state).shape: (13,)
state: [1000000

[I 2025-07-03 14:38:17,352] Trial 0 finished with value: 13.362418366639673 and parameters: {'learning_rate': 0.00017862234745427143, 'n_steps': 50, 'gamma': 0.9399658260932027, 'ent_coef': 0.007106783234186186}. Best is trial 0 with value: 13.362418366639673.


state: [1000000.0, np.float64(123.4706039428711), 0, np.float64(-4.648599978133859), np.float64(39.011473477232165), np.float64(-130.96362055980674), np.float64(35.95635234416873), np.float64(138.29393107096354), np.float64(140.56728642781576), np.float64(147.86027471441827), np.float64(121.09183405023016), np.float64(22.89999961853028), np.float64(29.95463800380079)]
np.array(state).shape: (13,)
Step reward: 0.00977536711501889
Current state shape: (13,)
Step reward: -0.025720364456181415
Current state shape: (13,)
Step reward: 0.08798758419875521
Current state shape: (13,)
Step reward: 0.013629059136961587
Current state shape: (13,)
Step reward: 0.014884192126465497
Current state shape: (13,)
Step reward: 0.0869878149078344
Current state shape: (13,)
Step reward: -0.0029436951477080585
Current state shape: (13,)
Step reward: 0.020483221966552084
Current state shape: (13,)
Step reward: 0.022896601885987914
Current state shape: (13,)
Step reward: -0.008620963745121845
Current state sha

...

state: [1000000.0, np.float64(123.4706039428711), 0, np.float64(-4.648599978133859), np.float64(39.011473477232165), np.float64(-130.96362055980674), np.float64(35.95635234416873), np.float64(138.29393107096354), np.float64(140.56728642781576), np.float64(147.86027471441827), np.float64(121.09183405023016), np.float64(22.89999961853028), np.float64(29.95463800380079)]
np.array(state).shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current sta

[I 2025-07-03 14:40:45,561] Trial 18 finished with value: 27.60640236133414 and parameters: {'learning_rate': 0.0006615388535469933, 'n_steps': 5, 'gamma': 0.9533340857791796, 'ent_coef': 7.314281110226074e-08}. Best is trial 18 with value: 27.60640236133414.


state: [1000000.0, np.float64(123.4706039428711), 0, np.float64(-4.648599978133859), np.float64(39.011473477232165), np.float64(-130.96362055980674), np.float64(35.95635234416873), np.float64(138.29393107096354), np.float64(140.56728642781576), np.float64(147.86027471441827), np.float64(121.09183405023016), np.float64(22.89999961853028), np.float64(29.95463800380079)]
np.array(state).shape: (13,)
Step reward: 0.01150043190002907
Current state shape: (13,)
Step reward: -0.02770465553284157
Current state shape: (13,)
Step reward: 0.09524111121367897
Current state shape: (13,)
Step reward: 0.014940552444453352
Current state shape: (13,)
Step reward: 0.022189920654299203
Current state shape: (13,)
Step reward: 0.13767072158813246
Current state shape: (13,)
Step reward: -0.004390952091978397
Current state shape: (13,)
Step reward: 0.07078134719848167
Current state shape: (13,)
Step reward: 0.07179490231323289
Current state shape: (13,)
Step reward: -0.05366395538330544
Current state shape: 

[I 2025-07-03 14:40:53,264] Trial 19 finished with value: 0.004537449334713165 and parameters: {'learning_rate': 0.0007292864376238639, 'n_steps': 10, 'gamma': 0.937209628283432, 'ent_coef': 2.320215004441568e-08}. Best is trial 18 with value: 27.60640236133414.


state: [1000000.0, np.float64(123.4706039428711), 0, np.float64(-4.648599978133859), np.float64(39.011473477232165), np.float64(-130.96362055980674), np.float64(35.95635234416873), np.float64(138.29393107096354), np.float64(140.56728642781576), np.float64(147.86027471441827), np.float64(121.09183405023016), np.float64(22.89999961853028), np.float64(29.95463800380079)]
np.array(state).shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current state shape: (13,)
Step reward: 0.0
Current sta

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
from ray import train as ray_train

import ray
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining

from stable_baselines3 import A2C
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from stable_baselines3.common.callbacks import BaseCallback

# Load and prepare data
df = pd.read_csv("/home/group3/processed_full.csv")
df['date'] = pd.to_datetime(df['date'])
df = df[df['tic'] == 'AAPL'].sort_values(by='date')
df.reset_index(drop=True, inplace=True)

# Ensure no NaN values
tech_indicator_list = [
    'macd', 'rsi_30', 'cci_30', 'dx_30',
    'close_30_sma', 'close_60_sma',
    'boll_ub', 'boll_lb', 'turbulence'
]

# Check for missing indicators
missing = [col for col in tech_indicator_list if col not in df.columns]
if missing:
    raise ValueError(f"Missing technical indicators in dataset: {missing}")

# Fill or drop NaNs
df.fillna(method="ffill", inplace=True)
df.dropna(inplace=True)


def create_env(data):
    data = data.copy().reset_index(drop=True)

    # Ensure the DataFrame is not empty
    if data.empty:
        raise ValueError("Provided DataFrame to create_env is empty.")

    stock_dim = len(data.tic.unique())
    state_space = 1 + 2 * stock_dim + len(tech_indicator_list) * stock_dim
    action_space = stock_dim
    num_stock_shares = [0] * stock_dim

    return StockTradingEnv(
        df=data,
        stock_dim=stock_dim,
        hmax=100,
        initial_amount=1e6,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=[0.001] * stock_dim,
        sell_cost_pct=[0.001] * stock_dim,
        reward_scaling=1e-4,
        state_space=state_space,
        action_space=action_space,
        tech_indicator_list=tech_indicator_list,
        user_defined_feature=[]
    )


class TuneReportCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)

    def _on_step(self) -> bool:
        if self.n_calls % 1000 == 0 and len(self.model.ep_info_buffer) >= 10:
            last_rewards = list(self.model.ep_info_buffer)[-10:]
            mean_reward = np.mean([ep['r'] for ep in last_rewards if 'r' in ep])
            ray_train.report({"total_reward": mean_reward, "training_iteration": self.num_timesteps})
        return True


def train_a2c(config):
    import traceback
    try:
        # Data split with fixed date formatting
        train_df = df[(df['date'] >= '2020-02-02') & (df['date'] <= '2022-12-31')].reset_index(drop=True)
        test_df  = df[(df['date'] >= '2023-01-01') & (df['date'] <= '2023-12-28')].reset_index(drop=True)

        env_train = create_env(train_df)
        env_test  = create_env(test_df)

        model = A2C(
            "MlpPolicy",
            env_train,
            learning_rate=config["learning_rate"],
            n_steps=config["n_steps"],
            gamma=config["gamma"],
            ent_coef=config["ent_coef"],
            vf_coef=config.get("vf_coef", 0.5),
            max_grad_norm=config.get("max_grad_norm", 0.5),
            gae_lambda=config.get("gae_lambda", 0.95),
            verbose=0,
            device="cpu"
        )

        model.learn(total_timesteps=10_000, callback=TuneReportCallback())

        reset_out = env_test.reset()
        obs = reset_out[0] if isinstance(reset_out, tuple) else reset_out

        total_reward = 0
        done = False
        while not done:
            action, _ = model.predict(obs)
            step_out = env_test.step(action)
            if len(step_out) == 5:
                obs, reward, terminated, truncated, info = step_out
                done = terminated or truncated
            else:
                obs, reward, done, info = step_out
            total_reward += reward

        print(f"[{datetime.datetime.now()}] Final test reward: {total_reward}", flush=True)
        ray_train.report({"total_reward": total_reward})

    except Exception:
        print("\U0001F6A8 Exception in train_a2c:\n", traceback.format_exc(), flush=True)
        raise


if __name__ == "__main__":
    ray.init(ignore_reinit_error=True)

    pbt = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=1,
        hyperparam_mutations={
            "learning_rate": lambda: np.random.uniform(1e-5, 1e-2),
            "n_steps": [5, 10, 20, 50],
            "gamma": lambda: np.random.uniform(0.9, 0.999),
            "ent_coef": lambda: 10 ** np.random.uniform(-8, -2),
            "vf_coef": lambda: np.random.uniform(0.1, 1.0),
            "max_grad_norm": lambda: np.random.uniform(0.3, 5.0),
            "gae_lambda": lambda: np.random.uniform(0.8, 1.0),
        }
    )

    analysis = tune.run(
        train_a2c,
        resources_per_trial={"cpu": 1},
        config={
            "learning_rate": tune.uniform(1e-5, 1e-2),
            "n_steps": tune.choice([5, 10, 20, 50]),
            "gamma": tune.uniform(0.9, 0.999),
            "ent_coef": tune.loguniform(1e-8, 1e-2),
            "vf_coef": tune.uniform(0.1, 1.0),
            "max_grad_norm": tune.uniform(0.3, 5.0),
            "gae_lambda": tune.uniform(0.8, 1.0),
        },
        num_samples=32,
        scheduler=pbt,
        stop={"training_iteration": 50},
        reuse_actors=True,
        metric="total_reward",
        mode="max"
    )

    print("Best hyperparameters found were:   ", analysis.best_config)
    print("Best total reward:              ", analysis.best_result["total_reward"])


/tmp/ipykernel_1148192/3641543642.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
2025-06-30 20:04:44,422	INFO worker.py:1684 -- Calling ray.init() again after it has already been called.
2025-06-30 20:04:44,423	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,total_reward
train_a2c_b3af7_00000,14.7737
train_a2c_b3af7_00001,2.61768
train_a2c_b3af7_00002,0.298049
train_a2c_b3af7_00003,-0.0091618


2025-06-30 20:04:54,596	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/group3/ray_results/train_a2c_2025-06-30_20-04-44' in 0.0033s.
2025-06-30 20:04:54,599	INFO tune.py:1041 -- Total run time: 10.18 seconds (10.16 seconds for the tuning loop).


Best hyperparameters found were:    {'learning_rate': 0.0051516766303610064, 'n_steps': 50, 'gamma': 0.9187677611590532, 'ent_coef': 8.349001727462869e-08, 'vf_coef': 0.6162179381220239, 'max_grad_norm': 2.231050842340027, 'gae_lambda': 0.9372004380248778}
Best total reward:               14.7736697


In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
from ray import train as ray_train

import ray
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining

from stable_baselines3 import A2C
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from stable_baselines3.common.callbacks import BaseCallback

# Load and prepare data
df = pd.read_csv("/home/group3/processed_full.csv")
df['date'] = pd.to_datetime(df['date'])
df = df[df['tic'] == 'AAPL'].sort_values(by='date')
df.reset_index(drop=True, inplace=True)

# Ensure no NaN values
tech_indicator_list = [
    'macd', 'rsi_30', 'cci_30', 'dx_30',
    'close_30_sma', 'close_60_sma',
    'boll_ub', 'boll_lb', 'turbulence', 'sentiment_score', 'vix'
]

# Check for missing indicators
missing = [col for col in tech_indicator_list if col not in df.columns]
if missing:
    raise ValueError(f"Missing technical indicators in dataset: {missing}")

# Fill or drop NaNs
df.fillna(method="ffill", inplace=True)
df.dropna(inplace=True)


def create_env(data):
    data = data.copy().reset_index(drop=True)

    # Ensure the DataFrame is not empty
    if data.empty:
        raise ValueError("Provided DataFrame to create_env is empty.")

    stock_dim = len(data.tic.unique())
    state_space = 1 + 2 * stock_dim + len(tech_indicator_list) * stock_dim
    action_space = stock_dim
    num_stock_shares = [0] * stock_dim

    return StockTradingEnv(
        df=data,
        stock_dim=stock_dim,
        hmax=100,
        initial_amount=1e6,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=[0.001] * stock_dim,
        sell_cost_pct=[0.001] * stock_dim,
        reward_scaling=1e-4,
        state_space=state_space,
        action_space=action_space,
        tech_indicator_list=tech_indicator_list,
        user_defined_feature=[]
    )


class TuneReportCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)

    def _on_step(self) -> bool:
        if self.n_calls % 1000 == 0 and len(self.model.ep_info_buffer) >= 10:
            last_rewards = list(self.model.ep_info_buffer)[-10:]
            mean_reward = np.mean([ep['r'] for ep in last_rewards if 'r' in ep])
            ray_train.report({"total_reward": mean_reward, "training_iteration": self.num_timesteps})
        return True


def train_a2c(config):
    import traceback
    try:
        # Data split
        train_df = df[(df['date'] >= '2020-02-02') & (df['date'] <= '2022-12-31')].reset_index(drop=True)
        test_df  = df[(df['date'] >= '2023-01-01') & (df['date'] <= '2023-12-28')].reset_index(drop=True)

        env_train = create_env(train_df)
        env_test  = create_env(test_df)

        model = A2C(
            "MlpPolicy",
            env_train,
            learning_rate=config["learning_rate"],
            n_steps=config["n_steps"],
            gamma=config["gamma"],
            ent_coef=config["ent_coef"],
            vf_coef=config.get("vf_coef", 0.5),
            max_grad_norm=config.get("max_grad_norm", 0.5),
            gae_lambda=config.get("gae_lambda", 0.95),
            verbose=0,
            device="cpu"
        )

        model.learn(total_timesteps=10_000)

        # Calculate training reward (average over buffer)
        train_reward = 0
        if len(model.ep_info_buffer) > 0:
            train_reward = np.mean([ep['r'] for ep in model.ep_info_buffer if 'r' in ep])

        # Evaluate on test set
        reset_out = env_test.reset()
        obs = reset_out[0] if isinstance(reset_out, tuple) else reset_out

        total_reward = 0
        done = False
        while not done:
            action, _ = model.predict(obs)
            step_out = env_test.step(action)
            if len(step_out) == 5:
                obs, reward, terminated, truncated, info = step_out
                done = terminated or truncated
            else:
                obs, reward, done, info = step_out
            total_reward += reward

        print(f"[{datetime.datetime.now()}] Final test reward: {total_reward} | Train reward: {train_reward}", flush=True)

        # Report both
        ray_train.report({
            "total_reward": total_reward,  # used for optimization
            "train_reward": train_reward   # optional info
        })

    except Exception:
        print("🚨 Exception in train_a2c:\n", traceback.format_exc(), flush=True)
        raise



if __name__ == "__main__":
    ray.init(ignore_reinit_error=True)

    pbt = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=1,
        hyperparam_mutations={
            "learning_rate": lambda: np.random.uniform(1e-5, 1e-2),
            "n_steps": [5, 10, 20, 50],
            "gamma": lambda: np.random.uniform(0.9, 0.999),
            "ent_coef": lambda: 10 ** np.random.uniform(-8, -2),
            "vf_coef": lambda: np.random.uniform(0.1, 1.0),
            "max_grad_norm": lambda: np.random.uniform(0.3, 5.0),
            "gae_lambda": lambda: np.random.uniform(0.8, 1.0),
        }
    )

    analysis = tune.run(
        train_a2c,
        resources_per_trial={"cpu": 1},
        config={
            "learning_rate": tune.uniform(1e-5, 1e-2),
            "n_steps": tune.choice([5, 10, 20, 50]),
            "gamma": tune.uniform(0.9, 0.999),
            "ent_coef": tune.loguniform(1e-8, 1e-2),
            "vf_coef": tune.uniform(0.1, 1.0),
            "max_grad_norm": tune.uniform(0.3, 5.0),
            "gae_lambda": tune.uniform(0.8, 1.0),
        },
        num_samples=32,
        scheduler=pbt,
        stop={"training_iteration": 50},
        reuse_actors=True,
        metric="total_reward",  # still optimizing for test reward
        mode="max"
    )

    best_config = analysis.best_config
    best_result = analysis.best_result

    print("\n🎯 Best hyperparameters found:")
    for k, v in best_config.items():
        print(f"  {k}: {v}")

    print("\n🏆 Best performance:")
    print(f"  Test Reward (total_reward): {best_result['total_reward']:.2f}")
    if 'train_reward' in best_result:
        print(f"  Train Reward (train_reward): {best_result['train_reward']:.2f}")

    # Optional: view all results sorted by test reward
    print("\n📊 Top 5 Trials:")
    df = analysis.results_df.sort_values("total_reward", ascending=False)
    print(df[['total_reward', 'train_reward']].dropna().head(5))



/tmp/ipykernel_1193774/959832286.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
2025-07-01 18:32:59,711	INFO worker.py:1684 -- Calling ray.init() again after it has already been called.
2025-07-01 18:32:59,712	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,total_reward,train_reward
train_a2c_0d08e_00000,0.0162806,3.71165
train_a2c_0d08e_00001,0.277092,0.20742
train_a2c_0d08e_00002,30.7083,56.1815
train_a2c_0d08e_00003,0.33654,63.7323
train_a2c_0d08e_00004,-0.00289557,13.5919
train_a2c_0d08e_00005,-0.0480388,0.587349
train_a2c_0d08e_00006,0,4.28851
train_a2c_0d08e_00007,18.2259,48.9923
train_a2c_0d08e_00008,2.01185,2.37041
train_a2c_0d08e_00009,0,6.18802


2025-07-01 18:33:10,811	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_a2c_0d08e_00000. Skip exploit for Trial train_a2c_0d08e_00005
2025-07-01 18:33:11,069	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_a2c_0d08e_00000. Skip exploit for Trial train_a2c_0d08e_00003
2025-07-01 18:33:11,221	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_a2c_0d08e_00007. Skip exploit for Trial train_a2c_0d08e_00001
2025-07-01 18:33:13,745	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_a2c_0d08e_00007. Skip exploit for Trial train_a2c_0d08e_00002
2025-07-01 18:33:17,582	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_a2c_0d08e_00004. Skip exploit for Trial train_a2c_0d08e_00009
2025-07-01 18:33:18,061	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_a2c_0d08e_00010. Skip exploit for Trial train_a2c_0d08e_00013
2025-07-01 18:33:18,757	INFO pbt.py:716 -- [pbt]: no checkpoint for trial train_a2c_0d08e_00007. Skip exploit for Trial train_a2c_0d08e_00011
2025-0


🎯 Best hyperparameters found:
  learning_rate: 0.007930200752995114
  n_steps: 5
  gamma: 0.9402488874458206
  ent_coef: 0.0006815558675725959
  vf_coef: 0.5573157709832984
  max_grad_norm: 2.538836058793725
  gae_lambda: 0.8367847301726301

🏆 Best performance:
  Test Reward (total_reward): 30.71
  Train Reward (train_reward): 56.18

📊 Top 5 Trials:
             total_reward  train_reward
trial_id                               
0d08e_00002     30.708335     56.181514
0d08e_00014     30.708335     42.065106
0d08e_00019     30.708335     67.840544
0d08e_00031     30.381498     62.105299
0d08e_00030     30.229944      9.035029


In [4]:
import os
import pandas as pd
import numpy as np
import datetime
import traceback
import uuid
import ray
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining
from ray import train as ray_train  # still needed for checkpoint_dir

from stable_baselines3 import A2C
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from stable_baselines3.common.callbacks import BaseCallback

import matplotlib.pyplot as plt

# --- Configurable parameters ---
TOTAL_TRAIN_TIMESTEPS = 10_000
REPORT_FREQ = 500
NUM_CV_FOLDS = 3
USE_GPU = False

# Load and prepare data
df = pd.read_csv("/home/group3/processed_full.csv")
df['date'] = pd.to_datetime(df['date'])
df = df[df['tic'] == 'AAPL'].sort_values(by='date')
df.reset_index(drop=True, inplace=True)

tech_indicator_list = [
    'macd', 'rsi_30', 'cci_30', 'dx_30',
    'close_30_sma', 'close_60_sma',
    'boll_ub', 'boll_lb','vix', 'turbulence','sentiment_score'
]

missing = [col for col in tech_indicator_list if col not in df.columns]
if missing:
    raise ValueError(f"Missing technical indicators in dataset: {missing}")

df.fillna(method="ffill", inplace=True)
df.dropna(inplace=True)

def create_env(data):
    data = data.copy().reset_index(drop=True)

    if data.empty:
        raise ValueError("Provided DataFrame to create_env is empty.")

    stock_dim = len(data.tic.unique())
    state_space = 1 + 2 * stock_dim + len(tech_indicator_list) * stock_dim
    action_space = stock_dim
    num_stock_shares = [0] * stock_dim

    return StockTradingEnv(
        df=data,
        stock_dim=stock_dim,
        hmax=100,
        initial_amount=1e6,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=[0.001] * stock_dim,
        sell_cost_pct=[0.001] * stock_dim,
        reward_scaling=1e-4,
        state_space=state_space,
        action_space=action_space,
        tech_indicator_list=tech_indicator_list,
        user_defined_feature=[]
    )

class TuneReportCallback(BaseCallback):
    def __init__(self, verbose=0, report_freq=REPORT_FREQ):
        super().__init__(verbose)
        self.report_freq = report_freq

    def _on_step(self) -> bool:
        if self.n_calls % self.report_freq == 0 and len(self.model.ep_info_buffer) > 0:
            last_rewards = list(self.model.ep_info_buffer)[-10:]
            rewards = [ep['r'] for ep in last_rewards if 'r' in ep]
            if len(rewards) > 0:
                mean_reward = np.mean(rewards)
                ray_train.report({"val_reward": mean_reward, "training_iteration": self.num_timesteps})
        return True

def sharpe_ratio(rewards):
    rewards = np.array(rewards)
    mean_ret = np.mean(rewards)
    std_ret = np.std(rewards)
    if std_ret == 0:
        return 0
    return (mean_ret / std_ret) * np.sqrt(252)

from sklearn.model_selection import TimeSeriesSplit

def cross_validate(model):
    """
    Runs cross-validation on validation splits of data.
    Evaluates the model in each fold environment and returns average
    validation reward and Sharpe ratio over folds.
    """
    val_start_date = '2022-07-01'
    val_end_date = '2022-12-31'
    
    # Filter validation data for folds (we'll split it by time)
    val_df = df[(df['date'] >= val_start_date) & (df['date'] <= val_end_date)].reset_index(drop=True)
    
    # Use TimeSeriesSplit for time-based folding
    tscv = TimeSeriesSplit(n_splits=NUM_CV_FOLDS)
    
    fold_rewards = []
    
    for fold_idx, (train_index, val_index) in enumerate(tscv.split(val_df)):
        fold_val_df = val_df.iloc[val_index].reset_index(drop=True)
        
        if fold_val_df.empty:
            print(f"Warning: Fold {fold_idx} validation data is empty. Skipping this fold.")
            continue
        
        # Create env for validation fold
        val_env = create_env(fold_val_df)
        
        obs = val_env.reset()
        obs = obs[0] if isinstance(obs, tuple) else obs
        
        done = False
        rewards = []
        
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            step_out = val_env.step(action)
            if len(step_out) == 5:
                obs, reward, terminated, truncated, info = step_out
                done = terminated or truncated
            else:
                obs, reward, done, info = step_out
                
            rewards.append(reward)
        
        fold_rewards.append(np.sum(rewards))
    
    # Compute average val reward and Sharpe ratio across folds
    if fold_rewards:
        avg_val_reward = np.mean(fold_rewards)
        val_sharpe = sharpe_ratio(fold_rewards)
    else:
        avg_val_reward = 0
        val_sharpe = 0
    
    print(f"Cross-validation results: avg_reward={avg_val_reward:.4f}, sharpe={val_sharpe:.4f}")
    return avg_val_reward, val_sharpe


def train_a2c(config):
    try:
        train_df = df[(df['date'] >= '2020-02-02') & (df['date'] <= '2022-06-30')].reset_index(drop=True)
        env_train = create_env(train_df)
        device = "cuda" if USE_GPU else "cpu"

        model = A2C(
            "MlpPolicy",
            env_train,
            learning_rate=config["learning_rate"],
            n_steps=config["n_steps"],
            gamma=config["gamma"],
            ent_coef=config["ent_coef"],
            vf_coef=config.get("vf_coef", 0.5),
            max_grad_norm=config.get("max_grad_norm", 0.5),
            gae_lambda=config.get("gae_lambda", 0.95),
            verbose=0,
            device=device
        )

        callback = TuneReportCallback(report_freq=REPORT_FREQ)

        trial_id = str(uuid.uuid4())[:8]
        base_trial_dir = f"manual_trial_{trial_id}"
        os.makedirs(base_trial_dir, exist_ok=True)

        total_steps = 0
        while total_steps < TOTAL_TRAIN_TIMESTEPS:
            model.learn(total_timesteps=REPORT_FREQ, reset_num_timesteps=False, callback=callback)
            total_steps += REPORT_FREQ

            # Run cross-validation here periodically
            val_reward, val_sharpe = cross_validate(model)

            # Save checkpoint
            checkpoint_dir = os.path.join(base_trial_dir, f"checkpoint_{total_steps}")
            os.makedirs(checkpoint_dir, exist_ok=True)
            checkpoint_path = os.path.join(checkpoint_dir, "model.zip")
            model.save(checkpoint_path)
            print(f"Checkpoint saved at step {total_steps} to {checkpoint_path}", flush=True)

            # Report real validation metrics to Tune
            ray_train.report(
                {
                    "val_reward": val_reward,
                    "val_sharpe": val_sharpe,
                    "training_iteration": total_steps
                },
                checkpoint=ray_train.Checkpoint.from_directory(checkpoint_dir)
            )

        # Final cross-validation after training ends
        val_reward, val_sharpe = cross_validate(model)

        # Compute final training reward (optional)
        train_reward = 0
        if len(model.ep_info_buffer) > 0:
            rewards = [ep['r'] for ep in model.ep_info_buffer if 'r' in ep]
            train_reward = np.mean(rewards) if rewards else 0

        tune.report(
            val_reward=val_reward,
            val_sharpe=val_sharpe,
            train_reward=train_reward,
        )

    except Exception:
        print(traceback.format_exc())
        raise

if __name__ == "__main__":
    ray.init(ignore_reinit_error=True)

    pbt = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=1,
        hyperparam_mutations={
            "learning_rate": lambda: np.random.uniform(1e-5, 1e-2),
            "n_steps": [5, 10, 20, 50],
            "gamma": lambda: np.random.uniform(0.9, 0.999),
            "ent_coef": lambda: 10 ** np.random.uniform(-8, -2),
            "vf_coef": lambda: np.random.uniform(0.1, 1.0),
            "max_grad_norm": lambda: np.random.uniform(0.3, 5.0),
            "gae_lambda": lambda: np.random.uniform(0.8, 1.0),
        }
    )

    analysis = tune.run(
        train_a2c,
        resources_per_trial={"cpu": 1, "gpu": 1 if USE_GPU else 0},
        config={
            "learning_rate": tune.uniform(1e-5, 1e-2),
            "n_steps": tune.choice([5, 10, 20, 50]),
            "gamma": tune.uniform(0.9, 0.999),
            "ent_coef": tune.loguniform(1e-8, 1e-2),
            "vf_coef": tune.uniform(0.1, 1.0),
            "max_grad_norm": tune.uniform(0.3, 5.0),
            "gae_lambda": tune.uniform(0.8, 1.0),
        },
        num_samples=16,
        scheduler=pbt,
        stop={"training_iteration": 50},
        metric="val_reward",
        mode="max",
        keep_checkpoints_num=2,
        checkpoint_score_attr="val_reward",
    )

    print("\n\U0001F50D Searching for best checkpoint...")

    for trial in analysis.trials:
        print(f"Trial: {trial.trial_id} at {trial.local_path}")
        import glob
        model_files = glob.glob(os.path.join(trial.local_path, "**", "model.zip"), recursive=True)
        print(f"  Found model.zip files: {model_files}")

    best_trial = analysis.get_best_trial("val_reward", mode="max")
    best_checkpoint = analysis.get_best_checkpoint(best_trial, metric="val_reward", mode="max")

    if best_checkpoint is None:
        import glob
        trial_logdir = best_trial.local_path
        model_files = glob.glob(os.path.join(trial_logdir, "**", "model.zip"), recursive=True)
        print("Fallback: found model files:", model_files)
        if model_files:
            model_path = model_files[0]
        else:
            raise FileNotFoundError("❌ No valid checkpoint or model.zip found!")
    else:
        checkpoint_dir = best_checkpoint.to_directory()
        model_path = os.path.join(checkpoint_dir, "model.zip")


    print(f"✅ Using checkpoint from: {model_path}")

    test_df = df[(df['date'] >= '2023-01-01') & (df['date'] <= '2023-12-28')].reset_index(drop=True)
    env_test = create_env(test_df)
    device = "cuda" if USE_GPU else "cpu"
    best_model = A2C.load(model_path, env=env_test, device=device)

    print("\U0001F680 Running model on test environment...")

# Capture starting total asset from env
if hasattr(env_test, 'total_asset'):
    begin_total_asset = env_test.total_asset
elif hasattr(env_test, 'portfolio_value'):
    begin_total_asset = env_test.portfolio_value
else:
    begin_total_asset = 1e6  # fallback default

obs = env_test.reset()
obs = obs[0] if isinstance(obs, tuple) else obs

done = False
step_rewards = []
total_reward = 0
portfolio_values = [begin_total_asset]
step_count = 0

while not done:
    action, _ = best_model.predict(obs)
    step_out = env_test.step(action)

    if len(step_out) == 5:
        obs, reward, terminated, truncated, info = step_out
        done = terminated or truncated
    else:
        obs, reward, done, info = step_out

    step_rewards.append(reward)
    total_reward += reward
    step_count += 1

    # Track portfolio value each step if available
    current_asset = None
    if info and 'total_asset' in info:
        current_asset = info['total_asset']
    elif hasattr(env_test, 'total_asset'):
        current_asset = env_test.total_asset
    elif hasattr(env_test, 'portfolio_value'):
        current_asset = env_test.portfolio_value
    
    if current_asset is not None:
        portfolio_values.append(current_asset)
    else:
        portfolio_values.append(portfolio_values[-1])  # repeat last if no update

# Final total asset
end_total_asset = portfolio_values[-1]

print(f"\nday: {step_count}, episode: 1")
print(f"begin_total_asset: {begin_total_asset:.2f}")
print(f"end_total_asset: {end_total_asset:.2f}")
print(f"total_reward: {total_reward:.2f}")

print(f"state: {obs}")
print(f"np.array(state).shape: {np.array(obs).shape}")

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(np.cumsum(step_rewards), label="Cumulative Reward", color="green")
plt.xlabel("Step")
plt.ylabel("Cumulative Reward")
plt.title("Test-Time Reward Trajectory (Best Model)")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(portfolio_values, label="Portfolio Value", color="blue")
plt.xlabel("Step")
plt.ylabel("Portfolio Value")
plt.title("Test-Time Portfolio Value Trajectory")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig("test_reward_and_portfolio_trajectory.png")
plt.show()

/tmp/ipykernel_1531171/1537352480.py:40: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
2025-07-06 16:22:47,162	INFO worker.py:1684 -- Calling ray.init() again after it has already been called.
2025-07-06 16:22:47,163	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/group3/venv/lib/python3.12/site-packages/ray/tune/tune.py:730: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
  warnings.warn(


Trial name,should_checkpoint,val_reward,val_sharpe
train_a2c_b075e_00000,True,0,0
train_a2c_b075e_00001,True,-1.39597,-8.73622
train_a2c_b075e_00002,True,0,0
train_a2c_b075e_00003,True,-0.153556,-8.73622
train_a2c_b075e_00004,True,0,0
train_a2c_b075e_00005,True,-1.39597,-8.73622
train_a2c_b075e_00006,True,-0.80966,-8.73622
train_a2c_b075e_00007,True,0,0
train_a2c_b075e_00008,True,0,0
train_a2c_b075e_00009,True,-1.39597,-8.73622


2025-07-06 16:22:58,759	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/group3/ray_results/train_a2c_2025-07-06_16-22-47' in 0.0081s.
2025-07-06 16:22:58,764	INFO tune.py:1041 -- Total run time: 11.60 seconds (11.58 seconds for the tuning loop).



🔍 Searching for best checkpoint...
Trial: b075e_00000 at /tmp/ray/session_2025-07-06_16-18-54_147038_1531171/artifacts/2025-07-06_16-22-47/train_a2c_2025-07-06_16-22-47/driver_artifacts/train_a2c_b075e_00000_0_ent_coef=0.0000,gae_lambda=0.9514,gamma=0.9860,learning_rate=0.0038,max_grad_norm=2.9738,n_steps=20,vf_coe_2025-07-06_16-22-47
  Found model.zip files: []
Trial: b075e_00001 at /tmp/ray/session_2025-07-06_16-18-54_147038_1531171/artifacts/2025-07-06_16-22-47/train_a2c_2025-07-06_16-22-47/driver_artifacts/train_a2c_b075e_00001_1_ent_coef=0.0000,gae_lambda=0.8681,gamma=0.9655,learning_rate=0.0038,max_grad_norm=3.1544,n_steps=10,vf_coe_2025-07-06_16-22-47
  Found model.zip files: []
Trial: b075e_00002 at /tmp/ray/session_2025-07-06_16-18-54_147038_1531171/artifacts/2025-07-06_16-22-47/train_a2c_2025-07-06_16-22-47/driver_artifacts/train_a2c_b075e_00002_2_ent_coef=0.0001,gae_lambda=0.9775,gamma=0.9934,learning_rate=0.0059,max_grad_norm=2.5490,n_steps=5,vf_coef_2025-07-06_16-22-47
  